In [28]:
# 🚀 Auto-setup: installs deps + configures CFBD access
import os
import sys
from pathlib import Path

# Find _auto_setup.py regardless of current working directory
_current = Path().resolve()
_auto_setup_path = None
for parent in [_current] + list(_current.parents):
    candidate = parent / "starter_pack" / "_auto_setup.py"
    if candidate.exists():
        _auto_setup_path = candidate
        # Add project root to sys.path (auto_setup.py will also do this)
        if str(parent) not in sys.path:
            sys.path.insert(0, str(parent))
        break

if _auto_setup_path and _auto_setup_path.exists():
    # Execute the file directly without changing directory
    with open(_auto_setup_path, 'r') as f:
        exec(f.read(), {'__file__': str(_auto_setup_path)})
else:
    # Fallback: try relative path
    try:
        with open("./_auto_setup.py", 'r') as f:
            exec(f.read(), {'__file__': './_auto_setup.py'})
    except FileNotFoundError:
        print("⚠️  Could not find _auto_setup.py. Please run from starter_pack directory.")


🔧 Notebook auto-setup complete:
  • Project root: /Users/stephen_bowman/Documents/GitHub/Script_Ohio_2.0/starter_pack
  • Requirements: /Users/stephen_bowman/Documents/GitHub/Script_Ohio_2.0/starter_pack/requirements.txt
  • Missing modules: none
  • Auto install performed: False


In [29]:
# 05_matchup_predictor.ipynb
# 🏈 Simple Game Outcome Predictor Using EPA & Success Rate

# 🛠 Requirements:
# - pandas, scikit-learn, matplotlib, seaborn (install via `pip install pandas scikit-learn matplotlib seaborn`)

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Import starter pack configuration system
import sys
from pathlib import Path
_config_dir = Path().resolve() / "config"
if str(_config_dir.parent) not in sys.path:
    sys.path.insert(0, str(_config_dir.parent))
from config.data_config import get_starter_pack_config

# Get configuration
config = get_starter_pack_config()
DATA_DIR = str(config.data_dir)

plt.style.use('fivethirtyeight')
plt.rcParams["figure.figsize"] = [15,8]


In [ ]:
# 📂 Load current year game results and season stats

# Load with low_memory=False to avoid dtype warnings
games = pd.read_csv(os.path.join(DATA_DIR, "games.csv"), low_memory=False)

# Try to load stats for current year, fallback to previous year if not available
stats_file = os.path.join(DATA_DIR, "advanced_season_stats", f"{config.current_year}.csv")
if not os.path.exists(stats_file):
    print(f"⚠️  Stats file for {config.current_year} not found, trying {config.current_year - 1}...")
    stats_file = os.path.join(DATA_DIR, "advanced_season_stats", f"{config.current_year - 1}.csv")
    if os.path.exists(stats_file):
        print(f"✅ Using {config.current_year - 1} stats")
        training_year = config.current_year - 1
    else:
        raise FileNotFoundError(f"Stats file not found for {config.current_year} or {config.current_year - 1}")
else:
    training_year = config.current_year

stats = pd.read_csv(stats_file)

# Filter to FBS-only matchups where both teams have data
# Use training_year for games if current year has no completed games
games_current = games[
    (games["season"] == training_year) & 
    (games["home_classification"] == 'fbs') & 
    (games["away_classification"] == 'fbs')
].copy()

# Check if we have games with scores for this year
has_scores = games_current["home_points"].notna() & games_current["away_points"].notna()
if has_scores.sum() == 0 and training_year == config.current_year:
    # If current year has no scores, use previous year for games
    print(f"⚠️  {training_year} has no completed games with scores.")
    print(f"   Using {training_year - 1} games for training (stats still from {training_year})")
    games_current = games[
        (games["season"] == training_year - 1) & 
        (games["home_classification"] == 'fbs') & 
        (games["away_classification"] == 'fbs')
    ].copy()
    # Verify we have scores now
    has_scores = games_current["home_points"].notna() & games_current["away_points"].notna()
    if has_scores.sum() == 0:
        raise ValueError(
            f"Neither {training_year} nor {training_year - 1} have games with scores. "
            "This notebook requires completed games with final scores."
        )

print(f"{len(games_current)} FBS vs. FBS games (season: {games_current['season'].iloc[0] if len(games_current) > 0 else 'N/A'})")

693 FBS vs. FBS games in 2025


In [31]:
# 🔗 Join game data with team stats for both teams

# Check required columns exist in stats
required_stats_cols = ["team", "offense_ppa", "defense_ppa", "offense_successRate", "defense_successRate"]
missing_cols = [col for col in required_stats_cols if col not in stats.columns]
if missing_cols:
    print(f"⚠️  Warning: Missing columns in stats: {missing_cols}")
    print(f"Available columns: {list(stats.columns)[:10]}...")
    # Try alternative column names
    if "offense_ppa" not in stats.columns and "offense_epa" in stats.columns:
        print("Using 'epa' instead of 'ppa'...")
        stats = stats.rename(columns={
            "offense_epa": "offense_ppa",
            "defense_epa": "defense_ppa"
        })
    else:
        raise ValueError(f"Cannot proceed: required columns missing: {missing_cols}")

# Prep stats - keep original for predictions later
stats_for_merge = stats[required_stats_cols].copy()

# Check for missing values in stats
print(f"Stats loaded: {len(stats_for_merge)} teams")
print(f"Teams with complete stats: {stats_for_merge[required_stats_cols[1:]].notna().all(axis=1).sum()}")

# Merge home/away team stats
games_stats = games_current.merge(
    stats_for_merge, 
    left_on="home_team", 
    right_on="team", 
    how="inner"
).rename(columns={
    "offense_ppa": "home_offense_ppa", 
    "defense_ppa": "home_defense_ppa",
    "offense_successRate": "home_offense_successRate", 
    "defense_successRate": "home_defense_successRate"
}).drop("team", axis=1)

print(f"After home team merge: {len(games_stats)} games")

games_stats = games_stats.merge(
    stats_for_merge, 
    left_on="away_team", 
    right_on="team", 
    how="inner"
).rename(columns={
    "offense_ppa": "away_offense_ppa", 
    "defense_ppa": "away_defense_ppa",
    "offense_successRate": "away_offense_successRate", 
    "defense_successRate": "away_defense_successRate"
}).drop("team", axis=1)

print(f"{len(games_stats)} games with full stat coverage")


Stats loaded: 136 teams
Teams with complete stats: 136
After home team merge: 693 games
693 games with full stat coverage


In [ ]:
# 🧠 Feature engineering: create stat differentials

print(f"Starting with {len(games_stats)} games after merge")

# Check if scores columns exist
if "home_points" not in games_stats.columns or "away_points" not in games_stats.columns:
    print("⚠️  Score columns not found. Available columns:")
    print([col for col in games_stats.columns if 'point' in col.lower() or 'score' in col.lower()])
    raise ValueError("home_points and away_points columns not found in merged data")

# Convert scores to numeric (handles string scores)
games_stats["home_points"] = pd.to_numeric(games_stats["home_points"], errors='coerce')
games_stats["away_points"] = pd.to_numeric(games_stats["away_points"], errors='coerce')

# Check score data availability
has_scores = games_stats["home_points"].notna() & games_stats["away_points"].notna()
print(f"Games with scores: {has_scores.sum()} out of {len(games_stats)}")

# Check data types and sample values
if len(games_stats) > 0:
    print(f"home_points dtype: {games_stats['home_points'].dtype}")
    sample_cols = ['home_team', 'away_team', 'home_points', 'away_points']
    available_cols = [col for col in sample_cols if col in games_stats.columns]
    if available_cols:
        print(f"Sample score data:\n{games_stats[available_cols].head()}")
    print(f"Games with valid numeric scores: {has_scores.sum()}")

# Filter to completed games only (where scores exist)
games_stats = games_stats[has_scores].copy()

if len(games_stats) == 0:
    print("\n⚠️  No games with scores found.")
    print("This usually means:")
    print("  1. The season hasn't started yet or games haven't been played")
    print("  2. The data file doesn't include completed games")
    print("  3. Scores are stored in a different column name")
    print("\n💡 Try using a previous year's data by modifying the config or using a different season.")
    raise ValueError("No completed games found. Cannot train model without game outcomes.")

# Check for missing stats before creating features
stat_cols = ["home_offense_ppa", "home_defense_ppa", "home_offense_successRate", "home_defense_successRate",
             "away_offense_ppa", "away_defense_ppa", "away_offense_successRate", "away_defense_successRate"]
missing_stats = games_stats[stat_cols].isna().any(axis=1)
print(f"Games with missing stats: {missing_stats.sum()}")

# Create feature differentials
games_stats["ppa_diff"] = games_stats["home_offense_ppa"] - games_stats["away_defense_ppa"]
games_stats["ppa_allowed_diff"] = games_stats["home_defense_ppa"] - games_stats["away_offense_ppa"]
games_stats["successRate_diff"] = games_stats["home_offense_successRate"] - games_stats["away_defense_successRate"]
games_stats["successRate_allowed_diff"] = games_stats["home_defense_successRate"] - games_stats["away_offense_successRate"]

# Target: did home team win?
games_stats["home_win"] = (games_stats["home_points"] > games_stats["away_points"]).astype(int)

# Remove rows with missing feature values
feature_cols = ["ppa_diff", "ppa_allowed_diff", "successRate_diff", "successRate_allowed_diff"]
games_stats = games_stats.dropna(subset=feature_cols)

print(f"\nGames with complete data: {len(games_stats)}")

if len(games_stats) == 0:
    raise ValueError(
        "No games with complete feature data. This may happen if:\n"
        "1. The season is not complete yet\n"
        "2. Advanced stats are not available for the current year\n"
        "3. Try using a previous year's data (e.g., 2024) for training"
    )

# Check class distribution
win_count = games_stats['home_win'].sum()
loss_count = len(games_stats) - win_count
print(f"Home wins: {win_count} ({games_stats['home_win'].mean():.1%})")
print(f"Home losses: {loss_count} ({(1 - games_stats['home_win'].mean()):.1%})")

# Validate both classes exist
if games_stats["home_win"].nunique() < 2:
    raise ValueError(
        f"Data contains only one class ({games_stats['home_win'].unique()[0]}). "
        "Need both wins and losses for classification. "
        "Try using a previous year's data or check if the season is complete."
    )

X = games_stats[feature_cols]
y = games_stats["home_win"]


Starting with 693 games after merge
Games with scores: 0
⚠️  No games with scores found. Checking data...
Sample games columns: ['id', 'season', 'season_type', 'week', 'start_date', 'start_time_tbd', 'neutral_site', 'conference_game', 'attendance', 'venue_id']
Games with home_points not null: 0
Sample home_points values: 105167   NaN
105168   NaN
105169   NaN
105171   NaN
105177   NaN
Name: home_points, dtype: float64


ValueError: No completed games found. The season may not be complete yet, or data may be missing.

In [ ]:
# 🤖 Train/test split and model training

# Use stratified split to ensure both classes are in train and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y  # Ensures both classes are represented in train and test
)

# Validate training set has both classes
if y_train.nunique() < 2:
    raise ValueError(f"Training set has only one class. Classes: {y_train.unique()}")

print(f"Training set: {len(X_train)} games")
print(f"  Home wins: {y_train.sum()} ({y_train.mean():.1%})")
print(f"Test set: {len(X_test)} games")
print(f"  Home wins: {y_test.sum()} ({y_test.mean():.1%})")

# Train model
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"\nModel Accuracy: {acc:.2%}")
print(f"Baseline (always predict majority class): {max(y_test.mean(), 1 - y_test.mean()):.2%}")


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: np.int64(0)

In [ ]:
# 📊 Confusion matrix

cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Loss", "Win"], yticklabels=["Loss", "Win"])
plt.title("Confusion Matrix: Home Team Win Prediction")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


In [ ]:
# 🔍 Predict a specific matchup (example)

def predict_matchup(home_team, away_team):
    """Predict win probability for a matchup between two teams."""
    # Check if teams exist in stats
    home_stats = stats[stats["team"] == home_team]
    away_stats = stats[stats["team"] == away_team]
    
    if home_stats.empty:
        print(f"❌ Error: {home_team} not found in stats")
        return None
    if away_stats.empty:
        print(f"❌ Error: {away_team} not found in stats")
        return None
    
    home = home_stats.iloc[0]
    away = away_stats.iloc[0]
    
    # Check for missing stats
    required_cols = ["offense_ppa", "defense_ppa", "offense_successRate", "defense_successRate"]
    if home[required_cols].isna().any():
        print(f"❌ Error: {home_team} has missing stats")
        return None
    if away[required_cols].isna().any():
        print(f"❌ Error: {away_team} has missing stats")
        return None

    data = {
        "ppa_diff": home["offense_ppa"] - away["defense_ppa"],
        "ppa_allowed_diff": home["defense_ppa"] - away["offense_ppa"],
        "successRate_diff": home["offense_successRate"] - away["defense_successRate"],
        "successRate_allowed_diff": home["defense_successRate"] - away["offense_successRate"]
    }

    input_df = pd.DataFrame([data])
    prob = model.predict_proba(input_df)[0][1]
    print(f"🏈 {home_team} vs {away_team}")
    print(f"   Win probability: {prob:.1%}")
    print(f"   Predicted outcome: {'Home win' if prob > 0.5 else 'Away win'}")
    return prob

# Example predictions
print("Example Matchups:\n")
predict_matchup("Michigan", "Ohio State")
print()
predict_matchup("Georgia", "Alabama")


In [ ]:
# ✅ Summary

# In this notebook, you:
# - Joined team stats with game outcomes
# - Built a basic logistic regression model using stat differentials
# - Predicted outcomes and evaluated model accuracy
# - Created a reusable function to simulate future matchups

# 🧪 Try This:
# - Add features like talent composite, pass/rush rate, tempo
# - Use different models (RandomForest, XGBoost)
# - Predict score differential instead of binary win/loss

# 🔗 Bridge to Model Pack

## 🚀 Next Steps: From Basic Prediction to Machine Learning

Congratulations! You've built a basic prediction model. Here's how to take it to the next level:

### What You Just Learned → ML Features

Your starter pack model uses:
- `ppa_diff` → In model pack, this becomes `home_adjusted_epa - away_adjusted_epa_allowed`
- `successRate_diff` → In model pack, these are `home_adjusted_success` and `away_adjusted_success`
- Basic stats → Model pack uses **86 opponent-adjusted features** for better accuracy

### Explore Weekly Training Data

See how your concepts become ML features:

```python
# Load weekly training data to see the 86-feature format
import pandas as pd

# Option 1: Using path utility directly (recommended)
from model_pack.utils.path_utils import get_weekly_training_file
weekly_path = get_weekly_training_file(week=1, season=2025)
weekly = pd.read_csv(weekly_path)

# Option 2: Using config helper (alternative)
# weekly_path = config.get_weekly_training_file(week=1, season=2025)
# weekly = pd.read_csv(weekly_path)

print(f"Week 1: {len(weekly)} games, {len(weekly.columns)} features")

# Compare your features to ML features
print("\nYour features → ML equivalent:")
print("  ppa_diff → home_adjusted_epa, away_adjusted_epa_allowed")
print("  successRate_diff → home_adjusted_success, away_adjusted_success")

# See the actual ML features
print("\nSample ML features:")
print(weekly[['home_adjusted_epa', 'away_adjusted_epa', 
              'home_adjusted_success', 'away_adjusted_success',
              'home_elo', 'away_elo', 'spread']].head())
```

### Try Model Pack

1. **Linear Regression Model**: `../model_pack/01_linear_regression_margin.ipynb`
   - Uses your concepts but with opponent-adjusted features
   - Predicts score margin instead of just win/loss
   - Better accuracy with 86 features!

2. **XGBoost Model**: `../model_pack/03_xgboost_win_probability.ipynb`
   - Advanced ML model using all 86 features
   - Provides win probabilities and confidence intervals

### Get Agent Guidance

Use the Learning Navigator Agent for personalized help:

```python
from agents.analytics_orchestrator import AnalyticsOrchestrator, AnalyticsRequest

orchestrator = AnalyticsOrchestrator()
request = AnalyticsRequest(
    user_id='your_id',
    query='Bridge me from matchup predictor to model pack',
    query_type='learning',
    parameters={'current_notebook': '05_matchup_predictor.ipynb'},
    context_hints={'role': 'data_scientist'}
)
response = orchestrator.process_analytics_request(request)
print(response.insights)
```

### Key Differences

| Starter Pack (This Notebook) | Model Pack |
|------------------------------|------------|
| 4 basic features | 86 opponent-adjusted features |
| 55% accuracy | 65-70% accuracy |
| Basic stats | Schedule-adjusted metrics |
| Win/loss prediction | Margin + win probability |
| Current season only | 2016-2025 (Week 5+) |

### Recommended Learning Path

1. ✅ **You Are Here**: Basic prediction with simple features
2. **Next**: Explore weekly training data (Week 1) to see feature format using `get_weekly_training_file()`
3. **Then**: Try `../model_pack/01_linear_regression_margin.ipynb`
4. **Finally**: Understand feature importance in `../model_pack/06_shap_interpretability.ipynb`

---

💡 **Tip**: The weekly training data files (accessed via `get_weekly_training_file()` utility) show exactly how
starter pack metrics become the 86 features used in ML models. Week 5+ data is used for
temporal validation (no future data leakage). Files are located in `data/training/weekly/` with automatic
fallback support for legacy locations.
